# Using SSH to Configure Host Name Resolution using /etc/hosts  

## Overview:

This example will guide you through creating /etc/hosts for a set of nodes 

## Prerequisites:

This tutorial assumes you have a Chameleon account and basic experience using the Chameleon CLI. Some experience using Jupyter Notebooks is also recommended.

Additional information:

- Getting started tutorial: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html
- Setting up the CLI: https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html

#### Setup Environment:
We are going to create a private network from scratch with 3 hosts: a server and two clients.

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@UC'

# Set the reservation
export RESERVATION_NAME='ConfigureEtcHosts'

# Set your chameleon keypair name and path to the private ssh key.
# You may need to ensure that the matching private key is in your jupyter notebook.
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/.ssh/${SSH_KEY_NAME}

## Network Information
# Set the names for the network, subnet, router, and switch. 
export NETWORK_NAME="myNetwork"
export SUBNET_NAME="mySubnet"
export ROUTER_NAME="myRouter"

# Set the subnet
export NETWORK_SUBNET_CIDR="192.168.100.0/24"

## Server information
export SERVER_NAME='myServer'
# Server's fixed IP inside our private network 
export SERVER_IP='192.168.100.10'

## Client information
export CLIENT_NAME='myClient'
#Set the number of clients to create. 
# First client will be called $CLIENT_NAME-1
# Last client will be called $CLIENT_NAME-$CLIENT_COUNT
export CLIENT_COUNT=2
# Client's private network IPs will be automatically generated (think DHCP)

# Add Chameleon custom blazar client
#sudo pip install -e git+https://github.com/ChameleonCloud/python-blazarclient.git@allocatable-vlans#egg=python-blazarclient


#### Make SSH keypair available in openstack
Before we create servers and reservations and networks, we do need to have the ssh keypair we want to use in openstack. So we ask openstack

In [ ]:
openstack keypair list

and look for the key whose name matches `$SSH_KEY_NAME` as defined above. If it is not there, you need to first create the keypair.

#### What if I already have a keypair I want to use in openstack?

Then skip the next steps and go straight to Create Reservations.

Next is to add it to the list of keypairs openstack know to be associated with your account. In reality it is just uploading the public key, which is what you really need to ssh into a host. As with the lease, we do need a [name][2] associated with this key pair:

[2]:https://docs.openstack.org/python-openstackclient/latest/cli/command-objects/keypair.html

In [ ]:
openstack keypair create --public-key $SSH_PRIVATE_KEY.pub  $SSH_KEY_NAME
openstack keypair list

#### Create Reservations

This lease includes three reservations.  One reservation for Skylake compute nodes. One reservation for a public floating IP address. One reservation for an isolated network. 

In [ ]:
START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${NETWORK_NAME}
blazar lease-create \
   --physical-reservation min=$((CLIENT_COUNT +1 )),max=$((CLIENT_COUNT +1 )),resource_properties='["=", "$node_type", "compute_skylake"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${NETWORK_NAME}",resource_properties='["==","$physical_network","physnet1"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${RESERVATION_NAME}

RESERVATION=`blazar lease-show --format value -c id ${RESERVATION_NAME}`
echo RESERVATION $RESERVATION

NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${RESERVATION_NAME} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo NODE_RESERVATION $NODE_RESERVATION

#### Verify if lease has been created.
Do not continue until `blazar lease-show` below replies with an `ACTIVE` status:

In [ ]:
blazar lease-show --format value -c status "${RESERVATION_NAME}"

#### Finish Creating Network:

An isolated network requires a subnet, router, and external gateway.

In [ ]:
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET_CIDR} \
                        --dhcp \
                        --network ${NETWORK_NAME} \
                        ${SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${ROUTER_NAME} ${SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${ROUTER_NAME}

echo Network ${NETWORK_NAME} is ready for nodes!

#### Create a Server

This server `${SERVER_NAME}` will be created in the background and use `$SERVER_IP` as its private IP. Remember it will still take around 10 minutes to be ready for business. However, since we are creating this server in the background we can do something else in the meantime.

In [ ]:
echo Creating Server with fixed IP ${SERVER_IP}
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${NETWORK_NAME},v4-fixed-ip=${SERVER_IP} \
                        ${SERVER_NAME}
                        

#### Add a Floating IP to the Server

In [ ]:
FLOATING_IP=`lease_list_floating_ips $RESERVATION`
echo FLOATING_IP $FLOATING_IP 
# Associate floating IP to server $SERVER_NAME
openstack server add floating ip $SERVER_NAME $FLOATING_IP 

#### Create Clients

`${CLIENT_NAME}-1` and `${CLIENT_NAME}-2` will be assigned arbitrary private IPs (DHCP) from the subnet. 

In [ ]:
echo Creating Clients with auto assigned fixed IPs 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${NETWORK_NAME} \
                        --min ${CLIENT_COUNT} \
                        --max ${CLIENT_COUNT} \
                        ${CLIENT_NAME}
                        

**Wait for the nodes to become active!** That means running

In [ ]:
openstack server list

until `${SERVER_NAME}`, `${CLIENT_NAME}-1`, and `${CLIENT_NAME}-2` have `Status = ACTIVE`. Only then proceed to the next step. This will take a while and `Status = BUILD` is not good enough.

#### Get the fixed IPs of the nodes

Example code that will fetch the fixed and floating IPs of the `${SERVER_NAME}`. Only the floating IP is needed in the remaining steps. 

In [ ]:
#Get the Server IPs

SERVER_FIXED_IP=`openstack server show --format value -c addresses ${SERVER_NAME} | tr -d ' ' | cut -d "=" -f 2 | cut -d "," -f 1`
SERVER_FLOATING_IP=`openstack server show --format value -c addresses ${SERVER_NAME} | tr -d ' ' | cut -d "=" -f 2 | cut -d "," -f 2`

echo $SERVER_NAME Fixed IP $SERVER_FIXED_IP
echo $SERVER_NAME Floating IP $SERVER_FLOATING_IP

#### Build a file that will be appended to all /etc/hosts files

The hosts file will be saved in the Jupyter notebook container and copied to the server and clients in later steps.

In [ ]:
TEMP_HOST_FILE='hostfile.tmp'
echo > $TEMP_HOST_FILE
#Create a string to append to the /etc/host files
echo ${SERVER_IP} ${SERVER_NAME} >> $TEMP_HOST_FILE

CLIENT_NAME='myClient'
for i in $(seq 1 ${CLIENT_COUNT}); do
  NODE_NAME=${CLIENT_NAME}'-'${i}
  NODE_IP=`openstack server show --format value -c addresses ${NODE_NAME} | tr -d ' ' | cut -d "=" -f 2 | cut -d "," -f 1`
  echo $NODE_IP $NODE_NAME >> $TEMP_HOST_FILE
done
cat $TEMP_HOST_FILE

#### Copy the hosts to file all nodes

We will use the public IP off the server to access all nodes. We will hop from the server to each client using ssh forwarding.

##### Start an ssh-agent and add your private key.

This is necessary so you can configure all nodes with only one floating IP address. You will need to hop from your server to each client using ssh.

In [ ]:
#Start the ssh forwarding agent and add your private key
ssh-agent
ssh-add $SSH_PRIVATE_KEY

Use scp to transfer the hosts file to the server and then copy the file to /etc/hosts.

In [ ]:
#Copy the host file to the server
scp -o "UserKnownHostsFile=/dev/null" \
    -o "StrictHostKeyChecking=no" \
    -i $SSH_PRIVATE_KEY \
    $TEMP_HOST_FILE cc@${SERVER_FLOATING_IP}:.

#Append the file to the server /etc/host file
ssh -t \
    -o "UserKnownHostsFile=/dev/null" \
    -o "StrictHostKeyChecking=no" \
    -i $SSH_PRIVATE_KEY \
    cc@${SERVER_FLOATING_IP} \
    "sudo sh -c 'cat $TEMP_HOST_FILE >> /etc/hosts'"

Hop through the server and transfer the host file from the server to each of the clients.  Then copy the host file to /etc/hosts on each client.

In [ ]:
#For each client, copy the host file from the server to the client and configure the /etc/hosts file
for i in $(seq 1 ${CLIENT_COUNT}); do
  NODE_NAME=${CLIENT_NAME}'-'${i}
  NODE_IP=`openstack server show --format value -c addresses ${NODE_NAME} | tr -d ' ' | cut -d "=" -f 2 | cut -d "," -f 1`
  echo $NODE_IP $NODE_NAME 
  
  #Copy the file from the server to the client
  ssh -A \
      -o "UserKnownHostsFile=/dev/null" \
      -o "StrictHostKeyChecking=no" \
      -i $SSH_PRIVATE_KEY \
      cc@${SERVER_FLOATING_IP} \
      "scp -o \"UserKnownHostsFile=/dev/null\" -o \"StrictHostKeyChecking=no\" $TEMP_HOST_FILE cc@${NODE_IP}:."
  
  #Append the file to the /etc/hosts file on the client
  ssh -A \
      -o "UserKnownHostsFile=/dev/null" \
      -o "StrictHostKeyChecking=no" \
      -i $SSH_PRIVATE_KEY \
      cc@${SERVER_FLOATING_IP} \
      "ssh -o \"UserKnownHostsFile=/dev/null\" -o \"StrictHostKeyChecking=no\"  cc@${NODE_IP} \"sudo sh -c 'cat $TEMP_HOST_FILE >> /etc/hosts'\""

done


At this point you have configured all of your hosts to have the same /etc/hosts file.  You can now reference all of your hosts using their names from within any of the hosts. 

#### Tear everything down
The tear down order is
1. Floating IP
1. Instances
1. Lease

Details on how to do each step are presented in the *Jupyter Orchestration* notebook.
